Aprendi SQLAlchmy com essa aula: 

<iframe width="560" height="315" src="https://www.youtube.com/embed/t4C1c62Z4Ag?si=x03yAe_0dd0Urk8U" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

# Core

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine(
    'sqlite://',
    echo=True # Mostra as queries executadas
)

engine

Engine(sqlite://)

In [4]:
engine.dialect

In [5]:
conection = engine.connect()
conection

In [6]:
conection.connection.dbapi_connection

In [ ]:
# conection.close()

In [7]:
# Poll: Guarda as conexões para serem reutilizadas, acelerando a execução
engine.pool

In [10]:
engine.pool.status()

'SingletonThreadPool id:2551857389952 size: 1'

In [ ]:
from sqlalchemy import text

with engine.connect() as con:
    with con.begin(): # Inicia uma transação, isso garante que o resultado seja consistente. Caso ocorra um erro em qualquer parte do código, toda a transação é desfeita
        con.execute(text('CREATE TABLE test (id INTEGER PRIMARY KEY, name VARCHAR)'))
        con.execute(text('INSERT INTO test (name) VALUES ("Teste")'))
        con.execute(text('INSERT INTO test (name) VALUES ("Teste 2")'))
        con.execute(text('INSERT INTO test (name) VALUES ("Teste 3")'))

In [14]:
with engine.connect() as con:
    rs = con.execute(text('SELECT * FROM test'))
    for row in rs:
        print(row)

(1, 'Teste')
(2, 'Teste 2')
(3, 'Teste 3')


In [ ]:
# O rs é um objeto do tipo Result. Voce pode pegar os dados de várias formas:
# rs.fetone() -> Retorna a primeira linha
# rs.fetcmany(x) / rs.partitions(x) -> Retorna x linhas
# rs.fetchall() / rs.all() -> Retorna todas as linhas
# rs.first() -> Retorna a primeira linha, mas se não houver nenhuma não dá erro	
# Existem muitos outros métodos, veja a documentação

In [16]:
import sqlalchemy as sa

In [17]:
metadata = sa.MetaData()
metadata

MetaData()

In [18]:
t = sa.Table('Comentarios', metadata,
            sa.Column('id', sa.Integer(), nullable=False),
            sa.Column('usuario', sa.String(50), nullable=False),
            sa.Column('comentario', sa.String(200), nullable=False),
            sa.Column('live_id', sa.Integer()),
            sa.Column('data', sa.DateTime(), nullable=False),
            sa.PrimaryKeyConstraint('id')
)

In [20]:
engine2 = create_engine('sqlite:///database.db')
metadata.create_all(engine2)

In [22]:
inspect = sa.inspect(engine2)
print(inspect.get_table_names())
print(inspect.get_columns('Comentarios'))

['Comentarios']
[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'primary_key': 1}, {'name': 'usuario', 'type': VARCHAR(length=50), 'nullable': False, 'default': None, 'primary_key': 0}, {'name': 'comentario', 'type': VARCHAR(length=200), 'nullable': False, 'default': None, 'primary_key': 0}, {'name': 'live_id', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'data', 'type': DATETIME(), 'nullable': False, 'default': None, 'primary_key': 0}]


In [24]:
t2 = sa.Table('Comentarios', metadata, autoload_with=engine2)
t2

Table('Comentarios', MetaData(), Column('id', Integer(), table=<Comentarios>, primary_key=True, nullable=False), Column('usuario', String(length=50), table=<Comentarios>, nullable=False), Column('comentario', String(length=200), table=<Comentarios>, nullable=False), Column('live_id', Integer(), table=<Comentarios>), Column('data', DateTime(), table=<Comentarios>, nullable=False), schema=None)

In [26]:
sql = sa.select(t2)
print(sql)

SELECT "Comentarios".id, "Comentarios".usuario, "Comentarios".comentario, "Comentarios".live_id, "Comentarios".data 
FROM "Comentarios"


In [27]:
with engine2.connect() as con:
    rs = con.execute(sql)
    for row in rs:
        print(row)

In [30]:
# O select é um builder, você pode encadear vários métodos para montar a query
sql = (
    sa.select(t2)
    .where(
        (t2.c.usuario == 'João Assaoka')
        | (t2.c.usuario == 'Maria Assaoka')
        & (t2.c.data > '2021-01-01')
    )    
    .order_by(t2.c.data)
)
print(sql)

SELECT "Comentarios".id, "Comentarios".usuario, "Comentarios".comentario, "Comentarios".live_id, "Comentarios".data 
FROM "Comentarios" 
WHERE "Comentarios".usuario = :usuario_1 OR "Comentarios".usuario = :usuario_2 AND "Comentarios".data > :data_1 ORDER BY "Comentarios".data


In [37]:
from datetime import datetime

sql = (
    sa.insert(t2)
    .values(
        id=1,
        usuario='João Assaoka',
        comentario='Teste',
        live_id=1,
        data=datetime.now()
    )
)
print(sql)

INSERT INTO "Comentarios" (id, usuario, comentario, live_id, data) VALUES (:id, :usuario, :comentario, :live_id, :data)


In [41]:
with engine2.connect() as con:
    with con.begin():
        con.execute(sql)

In [42]:
with engine2.connect() as con:
    rs = con.execute(sa.select(t2))
    for row in rs:
        print(row)

(1, 'João Assaoka', 'Teste', 1, datetime.datetime(2025, 1, 3, 14, 59, 17, 658346))


In [45]:
sql = (
    sa.update(t2)
    .where(t2.c.id == 1)
    .values(usuario='Daniela Musa')
)

print(sql)

UPDATE "Comentarios" SET usuario=:usuario WHERE "Comentarios".id = :id_1


In [44]:
with engine2.connect() as con:
    with con.begin():
        con.execute(sql)
    
    with con.begin():
        rs = con.execute(sa.select(t2))
        for row in rs:
            print(row)

(1, 'Daniela Musa', 'Teste', 1, datetime.datetime(2025, 1, 3, 14, 59, 17, 658346))


# ORM

In [54]:
from sqlalchemy import Column, Integer, String, DateTime, func
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, registry
from datetime import datetime

"""class Base (DeclarativeBase):
    pass

# Voce pode criar com ou sem notação de tipos
class Comentario(Base):
    __tablename__ = 'Comentarios'

    id: Mapped[int] = mapped_column(primary_key=True)
    usuario: Mapped[str]
    comentario: Mapped[str]
    live_id: Mapped[int]
    data: Mapped[datetime] = mapped_column(server_default=func.now())

# Sem notação de tipos
class Comentario(Base):
    __tablename__ = 'Comentarios'

    id = Column(Integer, primary_key=True)
    usuario = Column(String(50), nullable=False)
    comentario = Column(String(200), nullable=False)
    live_id = Column(Integer, nullable=False)
    data = Column(DateTime, server_default=func.now())"""
    

# Podemos ainda criar assim:
reg = registry()
@reg.mapped_as_dataclass
class Comentario:
    __tablename__ = 'Comentarios'

    id: Mapped[int] = mapped_column(init=False, primary_key=True)
    usuario: Mapped[str]
    comentario: Mapped[str]
    live_id: Mapped[int]
    data: Mapped[datetime] = mapped_column(init=False, server_default=func.now())

# Existe ainda uma outra forma chamada Automap, que mapeia automaticamente as tabelas do banco de dados para classes. Olhe a documentação para mais informações

In [55]:
reg.metadata.create_all(engine2)
# Se usar as outras, chamar o create_all do Base: Base.metadata.create_all(engine2)

In [56]:
from sqlalchemy.orm import Session

with Session(engine2) as s:
    result = s.scalar(sa.select(Comentario).where(Comentario.usuario == 'Daniela Musa'))
    print(result)

Comentario(id=1, usuario='Daniela Musa', comentario='Teste', live_id=1, data=datetime.datetime(2025, 1, 3, 14, 59, 17, 658346))


In [57]:
with Session(engine2) as s:
    result = s.scalar(sa.select(Comentario).where(Comentario.usuario == 'Daniela Musa'))
    
    # result é um objeto do tipo Comentario, podemos mexer nele como se fosse um objeto normal
    result.usuario = 'João Assaoka'
    s.commit()

with Session(engine2) as s:
    result = s.scalars(sa.select(Comentario))
    for r in result:
        print(r)

Comentario(id=1, usuario='João Assaoka', comentario='Teste', live_id=1, data=datetime.datetime(2025, 1, 3, 14, 59, 17, 658346))


In [ ]:
# É possível alterar todo o codigo para assincrono, basta mudar algumas importações. Caso queira saber mais, veja a documentação